In [9]:
import pandas as pd
import sqlite3 as sql 

from blm_activity_db import BlmActivityDb

In [22]:
blm_replies_query = \
"""
WITH BlmCommunities as (
    SELECT CommunityId
    FROM Community
    WHERE PeriodId = ? and Stance = 1
), CounterCommunities as (
    SELECT CommunityId
    FROM Community
    WHERE PeriodId = ? and Stance = -1
)
SELECT
    PeriodId,
    ReplyingAccountId,
    RepliedToAccountId,
    ReplyingCommunityId,
    RepliedToCommunityId,
    NumRetweets
FROM 
    InterCommunityReply cr
WHERE 
    PeriodId = ?
AND
    ReplyingCommunityId in BlmCommunities
AND
    RepliedToCommunityId in CounterCommunities"""
counter_replies_query = \
"""
WITH BlmCommunities as (
    SELECT CommunityId
    FROM Community
    WHERE PeriodId = ? and Stance = 1
), CounterCommunities as (
    SELECT CommunityId
    FROM Community
    WHERE PeriodId = ? and Stance = -1
)
SELECT
    PeriodId,
    ReplyingAccountId,
    RepliedToAccountId,
    ReplyingCommunityId,
    RepliedToCommunityId,
    NumRetweets
FROM 
    InterCommunityReply
WHERE 
    PeriodId = ?
AND
    ReplyingCommunityId in CounterCommunities
AND
    RepliedToCommunityId in BlmCommunities"""
cols = ["PeriodId", "ReplyingAccountId", "RepliedToAccountId", "ReplyingCommunityId", "RepliedToCommunityId", "NumTweets"]
def get_interstance_replies_for_period(period: int):
    db = BlmActivityDb()
    with db.conn as conn:
        cur = conn.cursor()
        cur.execute(blm_replies_query, (period, period, period))
        blm_replies = cur.fetchall()
        cur.execute(counter_replies_query, (period, period, period))
        counter_replies= cur.fetchall()
    activist_replies = pd.DataFrame(blm_replies, columns = cols) 
    counter_replies = pd.DataFrame(counter_replies, columns = cols)
    return activist_replies, counter_replies
    

In [24]:
activist_reply_df, counter_reply_df = None, None
first_period, last_period = 1, 6
for i in range(first_period, last_period + 1):
    activist_replies, counter_replies = get_interstance_replies_for_period(i)
    if activist_reply_df is None:
        activist_reply_df = activist_replies
        counter_reply_df = counter_replies
    else:
        activist_reply_df = pd.concat([activist_reply_df, activist_replies], ignore_index = True)
        counter_reply_df = pd.concat([counter_reply_df, counter_replies], ignore_index = True)


In [25]:
activist_reply_df

,PeriodId,ReplyingAccountId,RepliedToAccountId,ReplyingCommunityId,RepliedToCommunityId,NumTweets
0,1,121572025,249665794,0,12,1
1,1,121572025,2741492717,0,12,1
2,1,121572025,2810211210,0,12,1
3,1,121572025,2817644011,0,12,1
4,1,121572025,53446001,0,12,1
...,...,...,...,...,...,...
1257,3,97300546,2217007286,12,2,1
1258,3,97300546,2909731363,12,2,1
1259,3,975364111,195271137,3,2,1
1260,3,98221214,2269674122,57,2,1


In [26]:
counter_reply_df

,PeriodId,ReplyingAccountId,RepliedToAccountId,ReplyingCommunityId,RepliedToCommunityId,NumTweets
0,1,105214234,218113799,12,0,2
1,1,1249578344,21314814,12,25,1
2,1,1482990582,600397644,12,2,1
3,1,1562746416,2812497540,12,7,1
4,1,160599290,2191370758,12,5,1
...,...,...,...,...,...,...
1421,3,961523510,14235078,2,1,2
1422,3,96681919,311660957,2,1,1
1423,3,977420312,13393052,2,4,1
1424,3,993021234,17579643,2,1,1


In [27]:
activist_reply_df.PeriodId.unique(), counter_reply_df.PeriodId.unique()

(array([1, 2, 3], dtype=object), array([1, 2, 3], dtype=object))

In [30]:
query = "SELECT distinct PeriodId FROM InterCommunityRetweet"
with db.conn:
    cur = db.conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
[row[0] for row in rows]

[1, 2, 3, 4, 5, 6]